In [1]:
import os
import glob
import pickle
import sys  

sys.path.insert(0, '../py')
from graviti import *

import numpy as np
import pandas as pd
from sklearn import neighbors, datasets
from sklearn.neighbors import NearestNeighbors

from  matplotlib import pyplot
from matplotlib.colors import ListedColormap
import seaborn as sns
sns.set(style='white')
import umap

import collections
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.metrics import balanced_accuracy_score

In [22]:
from collections import Counter

from sklearn.datasets import load_iris
from sklearn.svm import LinearSVC, NuSVC
from sklearn.model_selection import train_test_split

from imblearn.datasets import make_imbalance
from imblearn.under_sampling import NearMiss
from imblearn.pipeline import make_pipeline
from imblearn.metrics import classification_report_imbalanced

print(__doc__)

Automatically created module for IPython interactive environment


In [4]:
# Load the 2d umap projection for BRCA 
umap_proj_filename = '../data/descriptor_withI.umap.csv' 
umap_xy = pd.read_csv(umap_proj_filename)
# Pase the sample id in order to compare with the subtype table
umap_xy['Sample.ID'] = umap_xy['sample'].str.rsplit('-',n=3,expand=True)[0]

In [5]:
# Load the molecular subtype table from literature
subtypes_filename = '../data/subtypes.csv'
subtypes = pd.read_csv(subtypes_filename,header=1)

In [6]:
# merge the two tables on the Sample.ID
df_merged = pd.merge(umap_xy, subtypes, on="Sample.ID").copy()
df_merged.drop('Unnamed: 0',axis=1,inplace=True)
df_merged.columns

Index(['x', 'y', 'sample', 'Sample.ID', 'Tumor.Type',
       'Included_in_previous_marker_papers', 'vital_status', 'days_to_birth',
       'days_to_death', 'days_to_last_followup',
       'age_at_initial_pathologic_diagnosis', 'pathologic_stage',
       'Tumor_Grade', 'BRCA_Pathology', 'BRCA_Subtype_PAM50', 'CESC_Pathology',
       'OV_Subtype', 'UCS_Histology', 'UCEC_Histology', 'MSI_status',
       'HPV_Status', 'tobacco_smoking_history', 'CNV Clusters',
       'Mutation Clusters', 'DNA.Methylation Clusters', 'mRNA Clusters',
       'miRNA Clusters', 'lncRNA Clusters', 'Protein Clusters',
       'PARADIGM Clusters', 'Pan-Gyn Clusters'],
      dtype='object')

In [7]:
# Define the feature set
feature_set = ['pathologic_stage','BRCA_Pathology', 'BRCA_Subtype_PAM50','CNV Clusters',
               'Mutation Clusters', 'DNA.Methylation Clusters', 'mRNA Clusters','miRNA Clusters',
               'lncRNA Clusters', 'Protein Clusters','PARADIGM Clusters', 'Pan-Gyn Clusters']

# Supervised learning of molecular features
We use the 2d UMAP projection to infer the annotated molecular subtypes

In [8]:
le = preprocessing.LabelEncoder()
feature_set = ['pathologic_stage','BRCA_Pathology', 'BRCA_Subtype_PAM50','CNV Clusters',
               'Mutation Clusters', 'DNA.Methylation Clusters', 'mRNA Clusters','miRNA Clusters',
               'lncRNA Clusters', 'Protein Clusters','PARADIGM Clusters', 'Pan-Gyn Clusters']

In [15]:
from imblearn.ensemble import BalancedRandomForestClassifier

for feature in ['BRCA_Subtype_PAM50']:#feature_set:
    X_full = df_merged[['x','y',feature]].copy()
    # Remove rows with missing target, separate target from predictors
    X_full.dropna(axis=0, subset=[feature], inplace=True)
    y = le.fit_transform(X_full[feature]) # label encode the target
    X_full.drop([feature], axis=1, inplace=True)

    print(__doc__)
    print(feature)
    
    RANDOM_STATE = 42

    # Create a folder to fetch the dataset
    X_train, X_test, y_train, y_test = train_test_split(X_full, y, random_state=RANDOM_STATE)

    print('Training target statistics: {}'.format(Counter(y_train)))
    print('Testing target statistics: {}'.format(Counter(y_test)))

    # Create a pipeline
    pipeline = make_pipeline(NearMiss(version=2),
                         LinearSVC(random_state=RANDOM_STATE))
    pipeline.fit(X_train, y_train)

    # Classify and report the results
    print(classification_report_imbalanced(y_test, pipeline.predict(X_test)))
    print('balanced accuracy: ',balanced_accuracy_score(y_test,pipeline.predict(X_test),adjusted=True))

Automatically created module for IPython interactive environment
BRCA_Subtype_PAM50
Training target statistics: Counter({2: 403, 3: 176, 0: 136, 1: 60, 4: 29})
Testing target statistics: Counter({2: 139, 3: 54, 0: 47, 1: 19, 4: 9})
                   pre       rec       spe        f1       geo       iba       sup

          0       0.50      0.19      0.96      0.28      0.43      0.17        47
          1       0.09      0.42      0.68      0.15      0.53      0.28        19
          2       0.00      0.00      1.00      0.00      0.00      0.00       139
          3       0.27      0.46      0.68      0.34      0.56      0.31        54
          4       0.04      0.33      0.75      0.08      0.50      0.24         9

avg / total       0.15      0.17      0.90      0.13      0.24      0.12       268

balanced accuracy:  0.10220957239434281


In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier

for feature in feature_set:
    X_full = df_merged[['x','y',feature]].copy()
    # Remove rows with missing target, separate target from predictors
    X_full.dropna(axis=0, subset=[feature], inplace=True)
    y = le.fit_transform(X_full[feature]) # label encode the target
    X_full.drop([feature], axis=1, inplace=True)

    brf = BalancedRandomForestClassifier(n_estimators=10, random_state=0)
    brf.fit(X_train, y_train) 

    y_pred = brf.predict(X_test)
    print(feature, balanced_accuracy_score(y_test,y_pred,adjusted=True)  )


In [78]:
from sklearn.model_selection import train_test_split as tts
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier as KNN

from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import (EditedNearestNeighbours,
                                     RepeatedEditedNearestNeighbours,
                                    RandomUnderSampler,
                                    NearMiss,
                                     NeighbourhoodCleaningRule)

def reporting(feature):
    print(__doc__)
    # Generate the dataset
    X_full = df_merged[['x','y',feature]].copy()
    # Remove rows with missing target, separate target from predictors
    X_full.dropna(axis=0, subset=[feature], inplace=True)
    y = le.fit_transform(X_full[feature]) # label encode the target
    X_full.drop([feature], axis=1, inplace=True)
    X = X_full.copy()
    # Create the samplers
    rus = RandomUnderSampler(random_state=0, replacement=True)
    nm = NearMiss(version=2)
    enn = EditedNearestNeighbours()
    ncr = NeighbourhoodCleaningRule()
    # Create the classifier
    knn = KNN(1)

    # Make the splits
    X_train, X_test, y_train, y_test = tts(X, y, random_state=42)

    # Add one transformers and two samplers in the pipeline object
    pipeline = make_pipeline(ncr, knn)

    pipeline.fit(X_train, y_train)
    y_hat = pipeline.predict(X_test)
    print('balanced accuracy: ',balanced_accuracy_score(y_test,pipeline.predict(X_test),adjusted=True))
    print(classification_report(y_test, y_hat))

In [ ]:
for feature in feature_set:
    print(feature)
    reporting(feature)

# Regrouping
Combine LumA/B to get better accuracy

In [51]:
X_subtype = df_merged[['x','y','BRCA_Subtype_PAM50']].copy()

def conditions(s): # define the grouping conditions
    if (s['BRCA_Subtype_PAM50'] == 'LumA') or (s['BRCA_Subtype_PAM50'] == 'LumB'): return 'L'
    elif s['BRCA_Subtype_PAM50'] == 'Her2': return 'H'
    elif s['BRCA_Subtype_PAM50'] == 'Basal': return 'B'
    elif s['BRCA_Subtype_PAM50'] == 'Normal': return 'N'

X_subtype['Class'] = X_subtype.apply(conditions, axis=1) # create the new grouped subtypes
#print(X_subtype.head())

# Remove rows with missing target, separate target from predictors
X_subtype.dropna(axis=0, subset=['Class'], inplace=True) 
y = le.fit_transform(X_subtype['Class']) # label encode the target
X_subtype.drop(['BRCA_Subtype_PAM50','Class'], axis=1, inplace=True)

print(__doc__)
print('Molecular subtypes')
    
RANDOM_STATE = 42

# Create a folder to fetch the dataset
X_train, X_test, y_train, y_test = train_test_split(X_subtype, y, random_state=RANDOM_STATE)

print('Training target statistics: {}'.format(Counter(y_train)))
print('Testing target statistics: {}'.format(Counter(y_test)))

# Create a pipeline
pipeline = make_pipeline(NearMiss(version=2),
#                         LinearSVC(random_state=RANDOM_STATE)
                         KNeighborsClassifier(n_neighbors=10)
                        )
pipeline.fit(X_train, y_train)

# Classify and report the results
print(classification_report_imbalanced(y_test, pipeline.predict(X_test)))
print('balanced accuracy: ',balanced_accuracy_score(y_test,pipeline.predict(X_test),adjusted=True))

Automatically created module for IPython interactive environment
Molecular subtypes
Training target statistics: Counter({2: 579, 0: 136, 1: 60, 3: 29})
Testing target statistics: Counter({2: 193, 0: 47, 1: 19, 3: 9})
                   pre       rec       spe        f1       geo       iba       sup

          0       0.18      0.19      0.81      0.19      0.39      0.15        47
          1       0.03      0.11      0.72      0.04      0.28      0.07        19
          2       0.54      0.07      0.85      0.12      0.24      0.05       193
          3       0.02      0.33      0.54      0.05      0.42      0.18         9

avg / total       0.42      0.10      0.83      0.12      0.28      0.07       268

balanced accuracy:  -0.10085221137214477


In [52]:
X_subtype = df_merged[['x','y','BRCA_Subtype_PAM50']].copy()
#X_subtype = X_subtype[X_subtype['BRCA_Subtype_PAM50'] != 'Normal']
#print(X_subtype.head())

# Remove rows with missing target, separate target from predictors
X_subtype.dropna(axis=0, subset=['BRCA_Subtype_PAM50'], inplace=True)
y = le.fit_transform(X_subtype['BRCA_Subtype_PAM50']) # label encode the target
X_subtype.drop(['BRCA_Subtype_PAM50'], axis=1, inplace=True)

print(__doc__)
print('Molecular subtypes')
    
RANDOM_STATE = 42

# Create a folder to fetch the dataset
X_train, X_test, y_train, y_test = train_test_split(X_subtype, y, random_state=RANDOM_STATE)

print('Training target statistics: {}'.format(Counter(y_train)))
print('Testing target statistics: {}'.format(Counter(y_test)))

# Create a pipeline
pipeline = make_pipeline(NearMiss(version=2),
                         KNeighborsClassifier(n_neighbors=10)
                        # LinearSVC(random_state=RANDOM_STATE)
                        )
pipeline.fit(X_train, y_train)

# Classify and report the results
print(classification_report_imbalanced(y_test, pipeline.predict(X_test)))
print('balanced accuracy: ',balanced_accuracy_score(y_test,pipeline.predict(X_test),adjusted=True))

Automatically created module for IPython interactive environment
Molecular subtypes
Training target statistics: Counter({2: 403, 3: 176, 0: 136, 1: 60, 4: 29})
Testing target statistics: Counter({2: 139, 3: 54, 0: 47, 1: 19, 4: 9})
                   pre       rec       spe        f1       geo       iba       sup

          0       0.22      0.17      0.87      0.19      0.39      0.14        47
          1       0.05      0.21      0.71      0.09      0.39      0.14        19
          2       0.61      0.10      0.93      0.17      0.31      0.09       139
          3       0.09      0.04      0.91      0.05      0.18      0.03        54
          4       0.03      0.33      0.58      0.05      0.44      0.19         9

avg / total       0.38      0.12      0.89      0.14      0.31      0.09       268

balanced accuracy:  -0.03704278085556942


In [45]:
list(zip(y,list(le.inverse_transform(y))))

[(2, 'L'),
 (0, 'B'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (0, 'B'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (0, 'B'),
 (3, 'N'),
 (0, 'B'),
 (2, 'L'),
 (0, 'B'),
 (1, 'H'),
 (2, 'L'),
 (3, 'N'),
 (2, 'L'),
 (0, 'B'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (0, 'B'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (1, 'H'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (0, 'B'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (3, 'N'),
 (0, 'B'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (0, 'B'),
 (0, 'B'),
 (2, 'L'),
 (3, 'N'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (1, 'H'),
 (0, 'B'),
 (1, 'H'),
 (2, 'L'),
 (0, 'B'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (1, 'H'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (0, 'B'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (0, 'B'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (2, 'L'),
 (1, 'H'),

In [38]:
X_subtype.shape

(1034, 2)